# Creating a Flask Signup Page: A Comprehensive Guide

This notebook will guide you through creating a Flask application with a signup, login and authentication basic functionality, following best practices and explaining each step in detail.

## 1. Setting up the Project Structure

First, let's create the project structure. You can create it throug UI of your favorite coding editor or Run the following commands in your terminal:

```bash
mkdir flask_signup_app
cd flask_signup_app
mkdir static templates
mkdir static/css static/js
touch app.py config.py models.py templates/signup.html static/css/styles.css
```

App file structure:
```terminal
├── app.py
├── config.py
├── models.py
├── static
│   ├── css
│   │   └── styles.css
│   └── js
└── templates
    └── signup.html
```

This structure follows Flask conventions and separates concerns:

- `app.py`: Main application file
- `config.py`: Configuration settings
- `models.py`: Database models
- `static/`: Static files (CSS, JavaScript)
- `templates/`: HTML templates

Benefits:

- Organized code structure improves maintainability
- Separation of concerns makes the application modular
- Follows Flask best practices, making it easier for other developers to understand

## 2. Creating the HTML Template (signup.html)

Create the signup form in `templates/signup.html`:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sign Up</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/styles.css') }}">
</head>
<body>
    <div class="container">
        <h1>Sign Up</h1>
        <form action="{{ url_for('signup') }}" method="POST">
            <div class="form-group">
                <label for="first_name">First Name:</label>
                <input type="text" id="first_name" name="first_name" required>
            </div>
            <div class="form-group">
                <label for="last_name">Last Name:</label>
                <input type="text" id="last_name" name="last_name" required>
            </div>
            <div class="form-group">
                <label for="email">Email:</label>
                <input type="email" id="email" name="email" required>
            </div>
            <div class="form-group">
                <label for="country">Country:</label>
                <input type="text" id="country" name="country" required>
            </div>
            <div class="form-group">
                <label for="username">Username:</label>
                <input type="text" id="username" name="username" required>
            </div>
            <div class="form-group">
                <label for="password">Password:</label>
                <input type="password" id="password" name="password" required>
            </div>
            <button type="submit">Sign Up</button>
        </form>
    </div>
</body>
</html>

Explanation:

- Uses semantic HTML5 structure
- Includes all required fields for signup
- Uses `url_for()` for linking CSS, which is a Flask best practice

Benefits:

- Semantic HTML improves accessibility and SEO
- Using `url_for()` makes the application more flexible and easier to maintain

## 3. Styling the Signup Page (styles.css)

Add the following CSS to `static/css/styles.css`:

In [ ]:
body {
    font-family: Arial, sans-serif;
    background-color: #f0f0f0;
    margin: 0;
    padding: 0;
}

.container {
    max-width: 400px;
    margin: 50px auto;
    background-color: #ffffff;
    padding: 20px;
    border-radius: 5px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
}

h1 {
    text-align: center;
    color: #333;
}

.form-group {
    margin-bottom: 15px;
}

label {
    display: block;
    margin-bottom: 5px;
    color: #666;
}

input[type="text"],
input[type="email"],
input[type="password"] {
    width: 100%;
    padding: 8px;
    border: 1px solid #ddd;
    border-radius: 4px;
    box-sizing: border-box;
}

button {
    width: 100%;
    padding: 10px;
    background-color: #007bff;
    color: #fff;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 16px;
}

button:hover {
    background-color: #0056b3;
}

## 4. Creating the Database Model (models.py)

Now, let's create the database model:

In [ ]:
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

db = SQLAlchemy()

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    first_name = db.Column(db.String(50), nullable=False)
    last_name = db.Column(db.String(50), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    country = db.Column(db.String(50), nullable=False)
    username = db.Column(db.String(50), unique=True, nullable=False)
    password = db.Column(db.String(255), nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)


def create_all_tables(app):
    with app.app_context():
        db.create_all()

Explanation:

- Uses SQLAlchemy ORM for database interactions
- Defines a User model with fields matching the signup form
- Includes created_at timestamp and auto-incrementing id

Benefits:

- ORM provides database independence
- Simplifies database operations and query construction
- Allows for easy database migrations and schema changes

## 5. Configuring the Application (config.py)

Let's set up the configuration:

In [ ]:
import os

basedir = os.path.abspath(os.path.dirname(__file__))

class Config:
    SECRET_KEY = 'my-secret-key'
    SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'app.db')
    SQLALCHEMY_TRACK_MODIFICATIONS = False

Explanation:

- Defines configuration as a class
- Uses environment variables with fallbacks
- Sets up SQLite as the default database

Benefits:

- Centralizes configuration, making it easier to manage
- Allows for different configurations (development, testing, production)
- Improves security by using environment variables for sensitive information

## 6. Creating the Flask Application (app.py)

Finally, let's create the main Flask application:

In [ ]:
from flask import Flask, render_template, request, flash, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash
from config import Config
from models import db, User

app = Flask(__name__)
app.config.from_object(Config)
db.init_app(app)

# Create all tables if they don't exist
create_all_tables(app)

@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        email = request.form['email']
        country = request.form['country']
        username = request.form['username']
        password = request.form['password']

        hashed_password = generate_password_hash(password)

        new_user = User(first_name=first_name, last_name=last_name, email=email,
                        country=country, username=username, password=hashed_password)

        try:
            db.session.add(new_user)
            db.session.commit()
            return redirect(url_for('signup'))
        except:
            db.session.rollback()

    return render_template('signup.html')

if __name__ == '__main__':
    app.run(debug=True)

Explanation:

- Initializes Flask app and database
- Creates database tables before the first request
- Defines signup route handling both GET and POST requests
- Hashes passwords for security
- Uses flash messages for user feedback

Benefits:

- Separation of concerns (routing, database operations)
- Secure password handling
- Provides user feedback for successful/failed operations

To run the application:

1. Install required packages: `pip install flask flask-sqlalchemy werkzeug`
2. Run the app: `python app.py`
3. Visit `http://localhost:5000/signup` in your browser


## 7. Extending App: Adding Login and Home Pages

Following modification will be done:

* Adding login.html and home.html pages in templates directory
* Modification in /static/css/styles.css file
* Modification in app.py file for importing sessions and check_hash_password functionality and adding routes for login, home and logout features.


### Add Login Page

* Create a login.html page in templates folder.

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Login</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/styles.css') }}">
</head>
<body>
    <div class="container">
        <h1>Login</h1>
        <form action="{{ url_for('login') }}" method="POST">
            <div class="form-group">
                <label for="username">Username:</label>
                <input type="text" id="username" name="username" required>
            </div>
            <div class="form-group">
                <label for="password">Password:</label>
                <input type="password" id="password" name="password" required>
            </div>
            <button type="submit">Login</button>
        </form>
        <p>Don't have an account? <a href="{{ url_for('signup') }}">Sign up</a></p>
    </div>
</body>
</html>

### Add Home Page

* Create a home.html page in templates folder.

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Home</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/styles.css') }}">
</head>
<body>
    <div class="container">
        <h1>Welcome, {{ username }}!</h1>
        <p>This is your home page.</p>
        <form action="{{ url_for('logout') }}" method="POST">
            <button type="submit">Logout</button>
        </form>
    </div>
</body>
</html>

#### Add following modification in the styles.css file.

In [ ]:
a {
    color: #007bff;
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}

### Modify app.py file

* Import required libraries
* Add login, home and logout routes

#### Following is the modified code of app.py file.

In [ ]:
from flask import Flask, render_template, request, flash, redirect, url_for, session
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash
from config import Config
from models import db, User, create_all_tables

app = Flask(__name__)
app.config.from_object(Config)
db.init_app(app)

# Create all tables if they don't exist
create_all_tables(app)

@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        email = request.form['email']
        country = request.form['country']
        username = request.form['username']
        password = request.form['password']

        hashed_password = generate_password_hash(password)

        new_user = User(first_name=first_name, last_name=last_name, email=email,
                        country=country, username=username, password=hashed_password)

        try:
            db.session.add(new_user)
            db.session.commit()
            flash('You have been successfully registered!', 'success')
            return redirect(url_for('login'))
        except:
            db.session.rollback()
            flash('Error: User registration failed. Please try again.', 'error')

    return render_template('signup.html')
    
    
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        
        user = User.query.filter_by(username=username).first()
        
        if user and check_password_hash(user.password, password):
            session['user_id'] = user.id
            flash('Logged in successfully!', 'success')
            return redirect(url_for('home'))
        else:
            flash('Invalid username or password', 'error')
    
    return render_template('login.html')

@app.route('/home')
def home():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    
    user = User.query.get(session['user_id'])
    return render_template('home.html', username=user.username)

@app.route('/logout', methods=['POST'])
def logout():
    session.pop('user_id', None)
    flash('You have been logged out', 'success')
    return redirect(url_for('login'))

if __name__ == '__main__':
    app.run(debug=True)

Explanation of new features:

1. Login Page: We've added a simple login form that accepts a username and password.

2. Home Page: A basic welcome page that displays the user's username and includes a logout button.

3. Updated CSS: We've added styles for links and ensured consistency across all pages.

4. New Routes in app.py:
   - `/login`: Handles user authentication.
   - `/home`: Displays the home page for logged-in users.
   - `/logout`: Logs out the user and redirects to the login page.

5. Session Management: We're using Flask's session to keep track of logged-in users.

6. Password Hashing: We're using `check_password_hash` to verify passwords during login.

To use these new features:

1. Ensure you have the new HTML templates in your `templates` folder.
2. Update your `static/css/styles.css` file with the new CSS.
3. Replace your existing `app.py` with the new version.
4. Run your Flask app: `python app.py`
5. Visit `http://localhost:5000/signup` to create an account, then you can log in at `http://localhost:5000/login`.

This extension adds basic user authentication to your Flask app, providing a more complete user experience with signup, login, and logout functionality.

### ---------------------------------------CyberLink Academy----------------------------------------------------------------
### ---------------------------------------Python Web Development--------------------------------------------------------